In [2]:
%matplotlib inline
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt

In [3]:
import numpy as np
import pandas as pd
import datetime as dt
import dateutil.relativedelta as rd

# Reflect Tables into SQLAlchemy ORM

In [4]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func

In [5]:
# create engine to hawaii.sqlite
engine = create_engine("sqlite:///Resources/hawaii.sqlite")

In [6]:
#census_data = pd.read_sql("SELECT * FROM Census_Data", conn)
# reflect an existing database into a new model
Base = automap_base()
# reflect the tables
Base.prepare(engine, reflect=True)

In [7]:
# View all of the classes that automap found
for Class in Base.classes.keys():
    print(Class)

measurement
station


In [8]:
# Save references to each table
Measurement = Base.classes.measurement
Station = Base.classes.station

In [9]:
# Create our session (link) from Python to the DB
session = Session(engine)

In [10]:
# some classes to display queries
def print_query(query):
    """
    Display all the variables in the item query.
    
        query -- A query, which in this case could be a class instance or a collection of class instances as a sqlalchemy.orm.query.Query.
    """
    if type(query) is sqlalchemy.orm.query.Query:
        for item in query:
            print_query(item)
    else:
        print(f"{'-'*4}class--{query.__class__.__name__}{'-'*4}")
        for key, value in query.__dict__.items():
            if not key.startswith("_"):
                print(f'{key} : {value}')

In [11]:
# Display some starter data
print_query(session.query(Measurement).first())
print_query(session.query(Station).limit(1))

----class--measurement----
tobs : 65.0
id : 1
prcp : 0.08
station : USC00519397
date : 2010-01-01
----class--station----
station : USC00519397
longitude : -157.8168
name : WAIKIKI 717.2, HI US
id : 1
elevation : 3.0
latitude : 21.2716


# Exploratory Precipitation Analysis

In [12]:
datetime_format = "%Y-%m-%d"

def to_datetime(value):
    """
    Convert the string into a datetime in a more compact function
    """
    # format using this template
    return dt.datetime.strptime(value, datetime_format)

# Find the most recent date in the data set.
most_recent_date = to_datetime(
    # Query the database on the class Measurement
    session.query(Measurement).\
        # order the column, use the decending argument
        order_by(Measurement.date.desc()).\
        # grab only the date column
        first().date)

most_recent_date

datetime.datetime(2017, 8, 23, 0, 0)

In [72]:
def measurement_query_to_dataframe(query):
    df = pd.read_sql(query.statement, session.bind)
    df.set_index('id', inplace = True)
    df['date'] = pd.to_datetime(df['date'], format=datetime_format)
    return df

# Design a query to retrieve the last 12 months of precipitation data and plot the results. 
# Starting from the most recent data point in the database. 

# Calculate the date one year from the last date in data set.
one_year_ago = most_recent_date - rd.relativedelta(years = 1)

# Perform a query to retrieve the data and precipitation scores
precipitation_scores = session.query(Measurement).\
    filter(
    Measurement.date >= one_year_ago.date(), 
    Measurement.date <= most_recent_date.date()
)
# Save the query results as a Pandas DataFrame and set the index to the date column
precipitation_scores_df = measurement_query_to_dataframe(precipitation_scores)
# Sort the dataframe by date
precipitation_scores_df.sort_values(by = ['date'], inplace = True)

# Use Pandas Plotting with Matplotlib to plot the data


In [14]:
# Use Pandas to calcualte the summary statistics for the precipitation data


# Exploratory Station Analysis

In [22]:
# Design a query to calculate the total number stations in the dataset
station_count = session.query(Measurement.station).distinct().count()
print(station_count)

9


In [47]:
# Design a query to find the most active stations (i.e. what stations have the most rows?)
# List the stations and the counts in descending order.
stations_activity = session.query(
    Measurement.station, 
    func.count(Measurement.station)
).group_by(Measurement.station).\
order_by(
    func.count(Measurement.station).desc()
).all()

stations_activity_df = pd.DataFrame({"station":[s for s,c in stations_activity],
                                     "count":[c for s,c in stations_activity]})
stations_activity_df.set_index('station', inplace = True)
stations_activity_df

,count
station,
USC00519281,2772
USC00519397,2724
USC00513117,2709
USC00519523,2669
USC00516128,2612
USC00514830,2202
USC00511918,1979
USC00517948,1372
USC00518838,511


In [89]:
# Using the most active station id from the previous query, calculate the lowest, highest, and average temperature.
most_active_station = stations_activity_df.iloc[0].name
station_history = session.query(Measurement).filter(Measurement.station == most_active_station)
station_history_df = measurement_query_to_dataframe(station_history)
station_history_df["tobs"].agg([min, max, "mean"])

min     54.000000
max     85.000000
mean    71.663781
Name: tobs, dtype: float64

In [94]:
# Using the most active station id
# Query the last 12 months of temperature observation data for this station
station_temperature_observation = session.query(Measurement).filter(
    Measurement.station == most_active_station,
    Measurement.date >= one_year_ago.date(), 
    Measurement.date <= most_recent_date.date()
)
station_temperature_observation_df = measurement_query_to_dataframe(station_temperature_observation)
station_temperature_observation_df
# plot the results as a histogram


,station,date,prcp,tobs
id,,,,
14608,USC00519281,2016-08-23,1.79,77.0
14609,USC00519281,2016-08-24,2.15,77.0
14610,USC00519281,2016-08-25,0.06,80.0
14611,USC00519281,2016-08-26,0.01,80.0
14612,USC00519281,2016-08-27,0.12,75.0
...,...,...,...,...
14955,USC00519281,2017-08-14,0.00,77.0
14956,USC00519281,2017-08-15,0.32,77.0
14957,USC00519281,2017-08-16,0.12,76.0


# Close session

In [66]:
# Close Session
session.close()